This just ellucidates and confirms that our effnet is doing what we expected

In [1]:
from model import *

import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset, train_test_loaders
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

ImportError: attempted relative import with no known parent package

In [11]:
hparams = argparse.Namespace(**{'load': False,
                            'window': 4096,
                            'step': 4096,
                             'classify': False,
                               'tgt_tax_lvl': "species",
                               'fwd_only': True,
                               'manifold': True})

In [7]:
path = 'global/homes/a/azaidi/scratch/azaidi/taxon_trials/models'
chunks = LazySeqDataset(hparams, 
                        path=path,
                       keep_open=True)

TypeError: __init__() got multiple values for argument 'path'

In [8]:
train_dl, val_dl, test_dl = train_test_loaders(chunks, batch_size=16, distances=True,
                                              downsample=0.1)
batch = next(iter(train_dl))
len(batch)

5

In [9]:
len(train_dl)

NameError: name 'train_dl' is not defined

In [9]:
len(train_dl)

951

Let's make sure to use the data/target that will match 

In [5]:
x = batch[1]
y = batch[2]
x.shape, y.shape

(torch.Size([16, 4096]), torch.Size([16, 16]))

In [8]:
model = EffNet_b0(out_feats=1, n_classes=16)

200
1


In [7]:
out = model(x.unsqueeze(1).to(torch.float))
out.shape, y.shape

(torch.Size([16, 16, 1]), torch.Size([16, 16]))

Looks like the forward pass does what's expected - now to just confirm with the loss function

In [ ]:
class DistMSELoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, output, target):
        x2 = output.pow(2).sum(axis=1)
        xy = 2*output.mm(output.T)
        dist = (((x2 - xy).T + x2))
        n = output.shape[0]
        loss = (dist - target).pow(2).sum()/(n*(n-1))
        return loss

In [12]:
DistMSELoss()(out.squeeze(2), y)

NameError: name 'DistMSELoss' is not defined